In [39]:
import bq_utils as bqu
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import arrow

In [40]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../auth/bq_key.json'
gh_archive = bqu.BigQueryHelper(active_project= "githubarchive", 
                                dataset_name = "day")
gh_archive

In [41]:
gh_archive.list_tables()[-10:]

['20190312',
 '20190313',
 '20190314',
 '20190315',
 '20190316',
 '20190317',
 '20190318',
 '20190319',
 '20190320',
 'yesterday']

In [4]:
gh_repo_links = open('../../../data/golang-repo-list.txt').readlines()
gh_repo_links = np.array([item.strip('\n').strip() for item in gh_repo_links])
gh_repo_links[:10], gh_repo_links.shape

(array(['https://github.com/urfave/cli',
        'https://github.com/mreiferson/go-httpclient',
        'https://github.com/crewjam/rfc5424',
        'https://github.com/kubernetes/heapster',
        'https://github.com/go-openapi/spec',
        'https://github.com/andygrunwald/go-gerrit',
        'https://github.com/openshift/ci-secret-mirroring-controller',
        'https://github.com/fsnotify/fsnotify',
        'https://github.com/BurntSushi/toml',
        'https://github.com/kubernetes-csi/drivers'], dtype='<U80'), (851,))

In [5]:
import re

pattern = re.compile(r'.*?github.com/(.*)', re.I)
repo_names = np.array(list(filter(None,[pattern.search(item).group(1) 
                                            if pattern.search(item) else None 
                                               for item in gh_repo_links])))
repo_names[:10], repo_names.shape

(array(['urfave/cli', 'mreiferson/go-httpclient', 'crewjam/rfc5424',
        'kubernetes/heapster', 'go-openapi/spec', 'andygrunwald/go-gerrit',
        'openshift/ci-secret-mirroring-controller', 'fsnotify/fsnotify',
        'BurntSushi/toml', 'kubernetes-csi/drivers'], dtype='<U61'), (845,))

# GitHub Issues\PRs count for last ten days

In [7]:
def add_query_params(query, params_dict):
    for i, j in params_dict.items():
        query = query.replace(i, j)
    return query

In [13]:
st = arrow.now().shift(days=-11)
et = arrow.now().shift(days=-1)
last_3_days = [dt.format('YYYYMMDD') for dt in arrow.Arrow.range('day', st, et)]
last_3_days, len(last_3_days)

(['20190301',
  '20190302',
  '20190303',
  '20190304',
  '20190305',
  '20190306',
  '20190307',
  '20190308',
  '20190309',
  '20190310',
  '20190311'],
 11)

In [10]:
year_prefix = '20*'
date_list = [item[2:] for item in last_3_days]
query_params = {
    '{year_prefix_wildcard}': year_prefix,
    '{year_suffix_month_day}': str(tuple(date_list)),
    '{repo_names}': str(tuple(repo_names))
}

In [11]:
query = """
SELECT  type, count(*)
        FROM `githubarchive.day.{year_prefix_wildcard}`
        WHERE _TABLE_SUFFIX IN {year_suffix_month_day}
        AND repo.name in {repo_names}
        AND type in ('PullRequestEvent', 'IssuesEvent')
        GROUP BY type
"""
query = add_query_params(query, query_params)
gh_archive.estimate_query_size(query)

0.6571597261354327

In [12]:
df = gh_archive.query_to_pandas(query)
df

,type,f0_
0,IssuesEvent,3237
1,PullRequestEvent,5430


# GitHub Issues\PRs count for 1-2 days around the time of past CVEs 

In [14]:
dates_of_interest = ['12-Oct-2018', '24-Jan-2019', '30-Mar-2017', '14-Dec-2018', '19-Nov-2018', '18-Dec-2018',
                     '30-Jan-2019', '12-May-2018', '26-Nov-2018', '13-Apr-2017', '23-Feb-2016', '2-Oct-2017',
                     '12-Jan-2016', '22-Oct-2018', '22-Oct-2018', '4-Aug-2016', '19-Dec-2018', '16-Feb-2018',
                     '2-Nov-2018', '9-Mar-2016', '3-Oct-2018', '29-Aug-2018', '13-Mar-2017', '9-Oct-2018',
                     '4-Jul-2018', '20-Dec-2018', '29-Jul-2018', '3-Feb-2018', '3-Jul-2018', '20-Nov-2018',
                     '18-Dec-2017', '24-Jul-2018', '25-Sep-2018', '10-Nov-2018', '4-Oct-2017', '24-Sep-2015',
                     '23-Feb-2016', '8-Nov-2018', '18-Jun-2018', '25-Mar-2016', '25-Sep-2018', '10-Feb-2018',
                     '14-Oct-2018', '7-Oct-2015', '13-Jul-2016', '2-Nov-2015', '27-Aug-2018', '26-Aug-2015',
                     '21-Oct-2016', '17-Mar-2016', '30-Jul-2015']
dates_of_interest = [record 
                         for dt_tuple in [(
                                            arrow.get(dt, 'D-MMM-YYYY').shift(days=-1).format('YYYYMMDD'),
                                            arrow.get(dt, 'D-MMM-YYYY').format('YYYYMMDD'),
                                            arrow.get(dt, 'D-MMM-YYYY').shift(days=+1).format('YYYYMMDD')
                                          ) for dt in dates_of_interest]
                         for record in dt_tuple]
dates_of_interest = np.unique(dates_of_interest)
dates_of_interest, dates_of_interest.shape

(array(['20150729', '20150730', '20150731', '20150825', '20150826',
        '20150827', '20150923', '20150924', '20150925', '20151006',
        '20151007', '20151008', '20151101', '20151102', '20151103',
        '20160111', '20160112', '20160113', '20160222', '20160223',
        '20160224', '20160308', '20160309', '20160310', '20160316',
        '20160317', '20160318', '20160324', '20160325', '20160326',
        '20160712', '20160713', '20160714', '20160803', '20160804',
        '20160805', '20161020', '20161021', '20161022', '20170312',
        '20170313', '20170314', '20170329', '20170330', '20170331',
        '20170412', '20170413', '20170414', '20171001', '20171002',
        '20171003', '20171004', '20171005', '20171217', '20171218',
        '20171219', '20180202', '20180203', '20180204', '20180209',
        '20180210', '20180211', '20180215', '20180216', '20180217',
        '20180511', '20180512', '20180513', '20180617', '20180618',
        '20180619', '20180702', '20180703', '201

In [15]:
year_prefix = '20*'
date_list = [item[2:] for item in dates_of_interest]
query_params = {
    '{year_prefix_wildcard}': year_prefix,
    '{year_suffix_month_day}': str(tuple(date_list)),
    '{repo_names}': str(tuple(repo_names))
}

In [16]:
query = """
SELECT  type, count(*)
        FROM `githubarchive.day.{year_prefix_wildcard}`
        WHERE _TABLE_SUFFIX IN {year_suffix_month_day}
        AND repo.name in {repo_names}
        AND type in ('PullRequestEvent', 'IssuesEvent')
        GROUP BY type
"""
query = add_query_params(query, query_params)
gh_archive.estimate_query_size(query)

5.574660442769527

In [17]:
df = gh_archive.query_to_pandas(query)
df

,type,f0_
0,PullRequestEvent,46875
1,IssuesEvent,30237


# BigQuery GitHub Data Retrieval (Issues & PRs)

In [18]:
dates_of_interest = ['12-Oct-2018', '24-Jan-2019', '30-Mar-2017', '14-Dec-2018', '19-Nov-2018', '18-Dec-2018',
                     '30-Jan-2019', '12-May-2018', '26-Nov-2018', '13-Apr-2017', '23-Feb-2016', '2-Oct-2017',
                     '12-Jan-2016', '22-Oct-2018', '22-Oct-2018', '4-Aug-2016', '19-Dec-2018', '16-Feb-2018',
                     '2-Nov-2018', '9-Mar-2016', '3-Oct-2018', '29-Aug-2018', '13-Mar-2017', '9-Oct-2018',
                     '4-Jul-2018', '20-Dec-2018', '29-Jul-2018', '3-Feb-2018', '3-Jul-2018', '20-Nov-2018',
                     '18-Dec-2017', '24-Jul-2018', '25-Sep-2018', '10-Nov-2018', '4-Oct-2017', '24-Sep-2015',
                     '23-Feb-2016', '8-Nov-2018', '18-Jun-2018', '25-Mar-2016', '25-Sep-2018', '10-Feb-2018',
                     '14-Oct-2018', '7-Oct-2015', '13-Jul-2016', '2-Nov-2015', '27-Aug-2018', '26-Aug-2015',
                     '21-Oct-2016', '17-Mar-2016', '30-Jul-2015']
dates_of_interest = [record 
                         for dt_tuple in [(
                                            arrow.get(dt, 'D-MMM-YYYY').shift(days=-1).format('YYYYMMDD'),
                                            arrow.get(dt, 'D-MMM-YYYY').format('YYYYMMDD'),
                                            arrow.get(dt, 'D-MMM-YYYY').shift(days=+1).format('YYYYMMDD')
                                          ) for dt in dates_of_interest]
                         for record in dt_tuple]
dates_of_interest = list(np.unique(dates_of_interest))
st = arrow.now().shift(days=-11)
et = arrow.now().shift(days=-1)
last_3_days = [dt.format('YYYYMMDD') for dt in arrow.Arrow.range('day', st, et)]
query_dates = dates_of_interest + last_3_days
len(query_dates), query_dates[:10]

(143,
 ['20150729',
  '20150730',
  '20150731',
  '20150825',
  '20150826',
  '20150827',
  '20150923',
  '20150924',
  '20150925',
  '20151006'])

In [19]:
year_prefix = '20*'
date_list = [item[2:] for item in query_dates]
query_params = {
    '{year_prefix_wildcard}': year_prefix,
    '{year_suffix_month_day}': str(tuple(date_list)),
    '{repo_names}': str(tuple(repo_names))
}

## Getting Issues

In [20]:
query = """
SELECT 
    repo.name as repo_name, 
    type as event_type, 
    actor.id as actor_id,
    actor.login as actor_name,
    JSON_EXTRACT_SCALAR(payload, '$.action') as issue_status,
    JSON_EXTRACT_SCALAR(payload, '$.issue.url') as issue_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.html_url') as issue_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.user.login') as issue_creator_name,
    JSON_EXTRACT_SCALAR(payload, '$.issue.user.url') as issue_creator_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.user.html_url') as issue_creator_url,
    JSON_EXTRACT_SCALAR(payload, '$.issue.comments') as comment_count,
    JSON_EXTRACT_SCALAR(payload, '$.issue.id') as issue_id,
    JSON_EXTRACT_SCALAR(payload, '$.issue.number') as issue_number,
    JSON_EXTRACT_SCALAR(payload, '$.issue.created_at') as issue_created_at,
    JSON_EXTRACT_SCALAR(payload, '$.issue.updated_at') as issue_updated_at,
    JSON_EXTRACT_SCALAR(payload, '$.issue.closed_at') as issue_closed_at,
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.issue.title'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as issue_title,
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.issue.body'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as issue_body
        
FROM `githubarchive.day.{year_prefix_wildcard}`
    WHERE _TABLE_SUFFIX IN {year_suffix_month_day}
    AND repo.name in {repo_names}
    AND type = 'IssuesEvent'
    """

query = add_query_params(query, query_params)
gh_archive.estimate_query_size(query)

399.72557612042874

In [21]:
issues_df = gh_archive.query_to_pandas(query)

In [22]:
issues_df.shape

(33474, 18)

In [23]:
issues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33474 entries, 0 to 33473
Data columns (total 18 columns):
repo_name                33474 non-null object
event_type               33474 non-null object
actor_id                 33474 non-null int64
actor_name               33474 non-null object
issue_status             33474 non-null object
issue_api_url            33474 non-null object
issue_url                33474 non-null object
issue_creator_name       33474 non-null object
issue_creator_api_url    33474 non-null object
issue_creator_url        33474 non-null object
comment_count            33474 non-null object
issue_id                 33474 non-null object
issue_number             33474 non-null object
issue_created_at         33474 non-null object
issue_updated_at         33474 non-null object
issue_closed_at          15563 non-null object
issue_title              33474 non-null object
issue_body               33466 non-null object
dtypes: int64(1), object(17)
memory usage: 4.6

In [24]:
issues_df.head()

,repo_name,event_type,actor_id,actor_name,issue_status,issue_api_url,issue_url,issue_creator_name,issue_creator_api_url,issue_creator_url,comment_count,issue_id,issue_number,issue_created_at,issue_updated_at,issue_closed_at,issue_title,issue_body
0,kubernetes/kubernetes,IssuesEvent,9358478,goltermann,closed,https://api.github.com/repos/kubernetes/kubern...,https://github.com/kubernetes/kubernetes/issue...,vishvananda,https://api.github.com/users/vishvananda,https://github.com/vishvananda,17,59309340,4914,2015-02-27T23:24:35Z,2015-08-26T21:00:57Z,2015-08-26T21:00:57Z,Clustered redis example fails to re-elect and ...,"Following the clustered redis example, everyth..."
1,influxdb/influxdb,IssuesEvent,219935,jwilder,closed,https://api.github.com/repos/influxdb/influxdb...,https://github.com/influxdb/influxdb/issues/3607,shilpisharma,https://api.github.com/users/shilpisharma,https://github.com/shilpisharma,42,100075419,3607,2015-08-10T14:06:42Z,2015-08-26T20:50:32Z,2015-08-26T20:50:32Z,[0.9.3-rc2] Unable to query influxdb,"Hi, We are doing feasibility testing. We inser..."
2,docker/docker,IssuesEvent,799078,cpuguy83,closed,https://api.github.com/repos/docker/docker/iss...,https://github.com/docker/docker/issues/9661,liuyunsh,https://api.github.com/users/liuyunsh,https://github.com/liuyunsh,14,51912295,9661,2014-12-14T07:12:24Z,2015-08-26T01:07:36Z,2015-08-26T01:07:36Z,Can not restart container after stopped contai...,Environment CentOS7 Kernel: [root@tengzhen ~]#...
3,kubernetes/kubernetes,IssuesEvent,9358478,goltermann,closed,https://api.github.com/repos/kubernetes/kubern...,https://github.com/kubernetes/kubernetes/issue...,Icedroid,https://api.github.com/users/Icedroid,https://github.com/Icedroid,25,94526384,11123,2015-07-12T02:28:36Z,2015-08-26T21:20:27Z,2015-08-26T21:20:27Z,Kubernetes 0.21 with flannel on CentOS 7,"I have two machines installed CentOS 7, I call..."
4,docker/docker,IssuesEvent,799078,cpuguy83,closed,https://api.github.com/repos/docker/docker/iss...,https://github.com/docker/docker/issues/8363,cpuguy83,https://api.github.com/users/cpuguy83,https://github.com/cpuguy83,14,44726927,8363,2014-10-02T19:09:52Z,2015-08-26T20:08:24Z,2015-08-26T20:08:24Z,PROPOSAL: docker volumes list command,### Problem Volumes remain a hidden piece with...


In [25]:
issues_df.issue_url.value_counts()[:10]

https://github.com/kubernetes/kubernetes/issues/56876    31
https://github.com/golang/go/issues/28673                 8
https://github.com/kubernetes/kubernetes/issues/70760     8
https://github.com/openshift/origin/issues/18826          8
https://github.com/kubernetes/test-infra/issues/11591     6
https://github.com/kubernetes/kubernetes/issues/53358     5
https://github.com/golang/go/issues/23867                 5
https://github.com/kubernetes/kubernetes/issues/15747     5
https://github.com/golang/go/issues/28948                 5
https://github.com/cockroachdb/cockroach/issues/18932     4
Name: issue_url, dtype: int64

In [26]:
issues_df.issue_created_at = pd.to_datetime(issues_df.issue_created_at)
issues_df.issue_updated_at = pd.to_datetime(issues_df.issue_updated_at)
issues_df.issue_closed_at = pd.to_datetime(issues_df.issue_closed_at)

In [27]:
issues_df = issues_df.loc[issues_df.groupby('issue_url').issue_updated_at.idxmax(skipna=False)]
issues_df.shape

(27109, 18)

In [28]:
issues_df.issue_url.value_counts()[:10]

https://github.com/cockroachdb/cockroach/issues/27183          1
https://github.com/istio/istio/issues/8268                     1
https://github.com/kubernetes/kubernetes/issues/67975          1
https://github.com/influxdata/influxdb/issues/12181            1
https://github.com/DataDog/dd-trace-go/issues/283              1
https://github.com/openshift/origin/issues/20841               1
https://github.com/openshift/oauth-proxy/issues/90             1
https://github.com/openshift/openshift-ansible/issues/11098    1
https://github.com/kubernetes/kubernetes/issues/34056          1
https://github.com/kubernetes/helm/issues/4300                 1
Name: issue_url, dtype: int64

In [29]:
issues_df.to_csv('../../../data/os-kube_gh-issues-12Mar.csv', index=False)

## Getting PRs

In [30]:
query = """
SELECT 
    repo.name as repo_name, 
    type as event_type, 
    actor.id as actor_id,
    actor.login as actor_name,
    JSON_EXTRACT_SCALAR(payload, '$.action') as pr_status,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.id') as pr_id,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.number') as pr_number,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.url') as pr_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.html_url') as pr_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.diff_url') as pr_diff_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.patch_url') as pr_patch_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.login') as pr_creator_name,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.url') as pr_creator_api_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.html_url') as pr_creator_url,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.created_at') as pr_created_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.updated_at') as pr_updated_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.closed_at') as pr_closed_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.merged_at') as pr_merged_at,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.merged') as pr_merged_status,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.comments') as pr_comments_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.review_comments') as pr_review_comments_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.commits') as pr_commits_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.additions') as pr_additions_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.deletions') as pr_deletions_count,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.changed_files') as pr_changed_files_count,    
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.pull_request.title'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as pr_title,
    TRIM(REGEXP_REPLACE(
             REGEXP_REPLACE(
                 JSON_EXTRACT_SCALAR(payload, '$.pull_request.body'), 
                 r'\\r\\n|\\r|\\n', 
                 ' '),
             r'\s{2,}', 
             ' ')) as pr_body
        
FROM `githubarchive.day.{year_prefix_wildcard}`
    WHERE _TABLE_SUFFIX IN {year_suffix_month_day}
    AND repo.name in {repo_names}
    AND type = 'PullRequestEvent'
"""

query = add_query_params(query, query_params)
gh_archive.estimate_query_size(query)

399.72557612042874

In [31]:
prs_df = gh_archive.query_to_pandas(query)

In [32]:
prs_df.shape

(52305, 27)

In [33]:
prs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52305 entries, 0 to 52304
Data columns (total 27 columns):
repo_name                   52305 non-null object
event_type                  52305 non-null object
actor_id                    52305 non-null int64
actor_name                  52305 non-null object
pr_status                   52305 non-null object
pr_id                       52305 non-null object
pr_number                   52305 non-null object
pr_api_url                  52305 non-null object
pr_url                      52305 non-null object
pr_diff_url                 52305 non-null object
pr_patch_url                52305 non-null object
pr_creator_name             52305 non-null object
pr_creator_api_url          52305 non-null object
pr_creator_url              52305 non-null object
pr_created_at               52305 non-null object
pr_updated_at               52305 non-null object
pr_closed_at                25866 non-null object
pr_merged_at                20477 non-null

In [34]:
prs_df.pr_url.value_counts()[:10]

https://github.com/openshift/release/pull/2752                               16
https://github.com/google/cadvisor/pull/909                                  12
https://github.com/kubernetes/kubernetes/pull/15124                           9
https://github.com/docker/docker/pull/16787                                   8
https://github.com/kubernetes/kubernetes/pull/13857                           8
https://github.com/openshift/release/pull/2748                                8
https://github.com/openshift/telemeter/pull/55                                8
https://github.com/kubernetes/heapster/pull/869                               7
https://github.com/operator-framework/operator-lifecycle-manager/pull/551     7
https://github.com/kubernetes/heapster/pull/1098                              6
Name: pr_url, dtype: int64

In [35]:
prs_df.pr_created_at = pd.to_datetime(prs_df.pr_created_at)
prs_df.pr_updated_at = pd.to_datetime(prs_df.pr_updated_at)
prs_df.pr_closed_at = pd.to_datetime(prs_df.pr_closed_at)
prs_df.pr_merged_at = pd.to_datetime(prs_df.pr_merged_at)

In [36]:
prs_df = prs_df.loc[prs_df.groupby('pr_url').pr_updated_at.idxmax(skipna=False)]
prs_df.shape

(36271, 27)

In [37]:
prs_df.pr_url.value_counts()[:10]

https://github.com/kubernetes/test-infra/pull/10907                              1
https://github.com/Azure/azure-sdk-for-go/pull/3110                              1
https://github.com/kubernetes/kubernetes/pull/44385                              1
https://github.com/cockroachdb/cockroach/pull/35608                              1
https://github.com/influxdata/influxdb/pull/5786                                 1
https://github.com/openshift/origin/pull/21917                                   1
https://github.com/cockroachdb/cockroach/pull/31089                              1
https://github.com/kubernetes/test-infra/pull/10523                              1
https://github.com/Azure/azure-sdk-for-go/pull/3101                              1
https://github.com/openshift/cluster-kube-controller-manager-operator/pull/69    1
Name: pr_url, dtype: int64

In [38]:
prs_df.to_csv('../../../data/os-kube_gh-pull_requests-12Mar.csv', index=False)